In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Change dir
%cd /content/drive/My\ Drive/car_classification

/content/drive/My Drive/car_classification


# Install packages

In [3]:
!cat requirements.txt

tensorflow-gpu==2.0.0
numpy
scipy
matplotlib
scikit-learn
jupyter
opencv-contrib-python
json_tricks
sklearn
git+https://github.com/qubvel/efficientnet

In [4]:
# !pip install -U git+https://github.com/qubvel/efficientnet
!pip install -U -r requirements.txt

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-v4_2rt23
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-v4_2rt23
     |████████████████████████████████| 380.8MB 44kB/s 
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 2)) (1.17.3)
Requirement already up-to-date: scipy in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 3)) (1.3.1)
Requirement already up-to-date: matplotlib in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 4)) (3.1.1)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 5)) (0.21.3)
Requirement already up-to-date: jupyter in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 6)) (1.0.0)
     |████████████████████████████████| 34.7MB 245kB/s 
Requirement already up-to-date: sklearn in /usr/local/lib/python3.6/dist-p

In [0]:
%matplotlib inline

# EfficientNetB0

In [0]:
!python main.py -ddir "../car_dataset" -af "../car_dataset/anotations.json" -e 50 \
                        -lr 1e-2 -kp 0.5 -b 32 -pl -dim 224 224

Directory './logs' was created successfully
Directory './models' was created successfully
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Number of examples: 767
X_train shape: (575, 224, 224, 3)
Y_train shape: (575, 15)
X_val shape: (192, 224, 224, 3)
Y_val shape: (192, 15)
2019-11-03 13:52:02.095276: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-03 13:52:02.117768: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-03 13:52:02.118617: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-11-03 13:52:02.118936: I tensorflow/stream_execu

In [0]:
!python main.py -ddir "../car_dataset" -af "../car_dataset/anotations.json" -e 50 \
                        -lr 1e-2 -kp 0.5 -b 32 -pl -dim 224 224 -off_lr_sch

Directory './logs' was created successfully
Directory './models' was created successfully
./models/model_2019_11_03-15_28_27.h5
./logs/run_2019_11_03-15_28_27
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Number of examples: 767
X_train shape: (575, 224, 224, 3)
Y_train shape: (575, 15)
X_val shape: (192, 224, 224, 3)
Y_val shape: (192, 15)
2019-11-03 15:28:48.270072: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-03 15:28:48.291832: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-03 15:28:48.292655: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusI

In [0]:
!python main.py -h

usage: main.py [-h] -ddir DATA_DIR [-wdir WEIGHTS_DIR] [-mdir MODEL_DIR]
               [-ldir LOG_DIR] [-af ANOT_FILE] [-dim DIMENSION DIMENSION]
               [-e NO_EPOCHS] [-lr LEARNING_RATE] [-kp KEEP_PROB] [-b BATCH]
               [-pl] [-fi] [-pt PATIENCE] [-bn {B0,B4}] [-off_lr_sch]

Car classification

optional arguments:
  -h, --help            show this help message and exit
  -ddir DATA_DIR, --data_dir DATA_DIR
                        Data directory
  -wdir WEIGHTS_DIR, --weights_dir WEIGHTS_DIR
                        Path to pretrained weights of model
  -mdir MODEL_DIR, --model_dir MODEL_DIR
                        Directory to save trained model (*.h5 file)
  -ldir LOG_DIR, --log_dir LOG_DIR
                        Directory to write logs and save files
  -af ANOT_FILE, --anot_file ANOT_FILE
                        Anotation file of car dataset (a dictionary includes
                        'boxes', 'scores' and 'scaled_area')
  -dim DIMENSION DIMENSION, --dimension D

In [8]:
!cat main.py

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout
from tensorflow.keras import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import load_model
from efficientnet.tfkeras import EfficientNetB0, EfficientNetB1, EfficientNetB4
from efficientnet.tfkeras import center_crop_and_resize, preprocess_input
import time
import os
import pydot
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot
import argparse
import cv2
from matplotlib import pyplot as plt
import json_tricks
from sklearn.model_selection import train_test_split

CLASS_NAMES = ['Chevrolet Orlando', 'Ford Ranger', 'Honda Civic', 'Honda CR-V', 'Hyundai Accent',\
     'Hyundai Grand i10', 'Kia Cerato', '

#EfficientNetB1

In [0]:
!python main.py -ddir "../car_dataset" -af "../car_dataset/anotations.json" -e 50 \
                        -lr 1e-2 -kp 0.2 -b 32 -pl -dim 224 224 -bn B1 -off_lr_sch

Directory './logs' was created successfully
Directory './models' was created successfully
./models/model_2019_11_03-16_12_13.h5
./logs/run_2019_11_03-16_12_13
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Number of examples: 767
X_train shape: (575, 224, 224, 3)
Y_train shape: (575, 15)
X_val shape: (192, 224, 224, 3)
Y_val shape: (192, 15)
2019-11-03 16:12:34.281607: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-03 16:12:34.308700: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-03 16:12:34.309597: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusI

In [0]:
!python main.py -ddir "../car_dataset" -af "../car_dataset/anotations.json" -e 50 \
                        -lr 1e-2 -kp 0.2 -b 32 -pl -dim 224 224 -bn B1

Directory './logs' was created successfully
Directory './models' was created successfully
./models/model_2019_11_03-16_34_25.h5
./logs/run_2019_11_03-16_34_25
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Number of examples: 767
X_train shape: (575, 224, 224, 3)
Y_train shape: (575, 15)
X_val shape: (192, 224, 224, 3)
Y_val shape: (192, 15)
2019-11-03 16:34:46.368312: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-03 16:34:46.392910: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-03 16:34:46.393707: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusI

# EfficientNetB2

In [10]:
!python main.py -h

usage: main.py [-h] -ddir DATA_DIR [-wdir WEIGHTS_DIR] [-mdir MODEL_DIR]
               [-ldir LOG_DIR] [-af ANOT_FILE] [-dim DIMENSION DIMENSION]
               [-e NO_EPOCHS] [-lr LEARNING_RATE] [-kp KEEP_PROB] [-b BATCH]
               [-pl] [-fi] [-pt PATIENCE] [-bn {B0,B1,B2,B4}] [-off_lr_sch]

Car classification

optional arguments:
  -h, --help            show this help message and exit
  -ddir DATA_DIR, --data_dir DATA_DIR
                        Data directory
  -wdir WEIGHTS_DIR, --weights_dir WEIGHTS_DIR
                        Path to pretrained weights of model
  -mdir MODEL_DIR, --model_dir MODEL_DIR
                        Directory to save trained model (*.h5 file)
  -ldir LOG_DIR, --log_dir LOG_DIR
                        Directory to write logs and save files
  -af ANOT_FILE, --anot_file ANOT_FILE
                        Anotation file of car dataset (a dictionary includes
                        'boxes', 'scores' and 'scaled_area')
  -dim DIMENSION DIMENSION, --dimen

In [0]:
!python main.py -ddir "../car_dataset" -af "../car_dataset/anotations.json" -e 50 \
                        -lr 1e-2 -kp 0.2 -b 32 -pl -dim 224 224 -bn B2

#EfficientNetB4

In [0]:
!python main.py -ddir "../car_dataset" -af "../car_dataset/anotations.json" -e 50 \
                        -lr 1e-2 -kp 0.2 -b 32 -pl -dim 224 224 -bn B4

Directory './logs' was created successfully
Directory './models' was created successfully
./models/model_2019_11_03-14_21_21.h5
./logs/run_2019_11_03-14_21_21
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Number of examples: 767
X_train shape: (575, 224, 224, 3)
Y_train shape: (575, 15)
X_val shape: (192, 224, 224, 3)
Y_val shape: (192, 15)
2019-11-03 14:21:42.531006: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-03 14:21:42.554161: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-03 14:21:42.555018: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusI

In [0]:
!python main.py -ddir "../car_dataset" -af "../car_dataset/anotations.json" -e 30 \
                        -lr 1e-2 -kp 0.2 -b 32 -pl -dim 224 224 -bn B4 -pt 20 -off_lr_sch

Directory './logs' was created successfully
Directory './models' was created successfully
./models/model_2019_11_03-15_43_38.h5
./logs/run_2019_11_03-15_43_38
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Number of examples: 767
X_train shape: (575, 224, 224, 3)
Y_train shape: (575, 15)
X_val shape: (192, 224, 224, 3)
Y_val shape: (192, 15)
2019-11-03 15:43:59.725451: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-03 15:43:59.746835: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-03 15:43:59.747598: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusI

# Fine-tuning

In [0]:
!python main.py -ddir "../car_dataset" -af "../car_dataset/anotations.json" -e 50 \
                        -wdir "./models/model_2019_11_03-16_34_25.h5" \
                        -lr 1e-2 -kp 0.2 -b 32 -pl -dim 224 224 -bn B1 -fi

Directory './logs' was created successfully
Directory './models' was created successfully
./models/model_2019_11_03-17_14_14.h5
./logs/run_2019_11_03-17_14_14
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Number of examples: 767
X_train shape: (575, 224, 224, 3)
Y_train shape: (575, 15)
X_val shape: (192, 224, 224, 3)
Y_val shape: (192, 15)
2019-11-03 17:14:35.085263: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-03 17:14:35.109838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-03 17:14:35.110612: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusI

In [9]:
!python main.py -ddir "../car_dataset" -af "../car_dataset/anotations.json" -e 50 \
                        -wdir "./models/model_2019_11_03-16_34_25.h5" \
                        -lr 1e-3 -kp 0.2 -b 32 -pl -dim 224 224 -bn B1 -fi

Directory './logs' was created successfully
Directory './models' was created successfully
./models/model_2019_11_04-04_57_29.h5
./logs/run_2019_11_04-04_57_29
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
Number of examples: 767
X_train shape: (575, 224, 224, 3)
Y_train shape: (575, 15)
X_val shape: (192, 224, 224, 3)
Y_val shape: (192, 15)
2019-11-04 04:57:50.326702: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-04 04:57:50.350509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-04 04:57:50.351238: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusI

In [0]:
!python main.py -ddir "../car_dataset" -af "../car_dataset/anotations.json" -e 50 \
                        -wdir "./models/model_2019_11_03-16_34_25.h5" \
                        -lr 1e-3 -kp 0.2 -b 32 -pl -dim 224 224 -bn B1 -fi

Directory './logs' was created successfully
Directory './models' was created successfully
./models/model_2019_11_04-06_02_34.h5
./logs/run_2019_11_04-06_02_34
libpng warning: iCCP: known incorrect sRGB profile


In [12]:
!cat main.py

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout
from tensorflow.keras import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import load_model
from efficientnet.tfkeras import EfficientNetB0, EfficientNetB1, EfficientNetB4
from efficientnet.tfkeras import center_crop_and_resize, preprocess_input
import time
import os
import pydot
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot
import argparse
import cv2
from matplotlib import pyplot as plt
import json_tricks
from sklearn.model_selection import train_test_split

CLASS_NAMES = ['Chevrolet Orlando', 'Ford Ranger', 'Honda Civic', 'Honda CR-V', 'Hyundai Accent',\
     'Hyundai Grand i10', 'Kia Cerato', '